##### 改ざん画像データセットのダウンロード・解凍
- 前回の試行の続きを行いたい場合（再開モードの場合）は実行不要です．
- これは本来のDEFACTOではなく，物体削除系の改ざん画像のみを取り出した簡易版です．
- 改ざん領域を含む128x128ピクセルの部分を切り出して使用しています（元々の改ざん領域のサイズが128x128より大きいものは無視しました）．
- ファイルサイズが大きいので，必要になるまでダウンロードしないほうが良いです．
- 数十分～1時間程度かかる可能性があります．

In [ ]:
import os
import torch
from torchvision import transforms
from mylib.data_io import CSVBasedDataset


def make_defacto_tensors(filename, dirname='./defacto'):
    transform = transforms.Grayscale()
    dataset = CSVBasedDataset(dirname=dirname, filename=os.path.join(dirname, filename), items=['Input Image', 'Ground Truth'], dtypes=['image', 'image'])
    data = [dataset[i] for i in range(len(dataset))]
    x = torch.cat([torch.unsqueeze(u, dim=0) for u, v in data], dim=0)
    y = torch.cat([torch.unsqueeze(transform(v), dim=0) for u, v in data], dim=0)
    del dataset, data
    return x, y

if not os.path.isfile('defacto_train_input_images.pt'):

    !wget "https://tus.box.com/shared/static/v4giouhhlttk29vgoaxuf56rsuzcjkc2.gz" -O defacto.tar.gz
    !tar -zxf defacto.tar.gz
    !rm -f defacto.tar.gz
    image_tensor, label_tensor = make_defacto_tensors(filename='train_list.csv')
    torch.save(image_tensor, './Datasets/defacto_train_input_images.pt')
    torch.save(label_tensor, './Datasets/defacto_train_target_images.pt')
    del image_tensor, label_tensor
    image_tensor, label_tensor = make_defacto_tensors(filename='test_list.csv')
    torch.save(image_tensor, './Datasets/defacto_test_input_images.pt')
    torch.save(label_tensor, './Datasets/defacto_test_target_images.pt')
    torch.save(image_tensor, './temp/defacto_test_input_images.pt')
    torch.save(label_tensor, './temp/defacto_test_target_images.pt')
    del image_tensor, label_tensor

    !rm -fr defacto

##### データセットの場所やバッチサイズなどの定数値の設定

In [2]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
import torch


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる．
# なお，Colab環境で再開モードを利用する場合は，前回終了時に temp ディレクトリの中身を自分の Google Drive に退避しておき，
# それを改めて /content/AI_advanced/temp 以下にあらかじめ移しておく必要がある．
RESTART_MODE = False


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする．
# GPU が複数存在する環境では，'cuda:0', 'cuda:1', 'cuda:2' などのような形で使用するGPUのIDを指定する．
# Google Colab, Paperspace Gradient などで GPU を利用する場合は DEVICE = 'cuda:0' とすれば良いはず．
DEVICE = 'cuda:0'

# 高速化・省メモリ化のために半精度小数を用いた混合精度学習を行うか否か（Trueの場合は行う）
USE_AMP = True
FLOAT_DTYPE = torch.float16 # 混合精度学習を行う場合の半精度小数の型．環境によっては torch.bfloat16 にした方が良好な性能になる（ただしColabのT4 GPU環境ではムリ）．

# 混合精度学習の設定
if DEVICE == 'cpu':
    USE_AMP = False # CPU使用時は強制的に混合精度学習をOFFにする
LOSS_SCALER = torch.amp.grad_scaler.GradScaler(enabled=USE_AMP, device='cuda', init_scale=2**16)
ADAM_EPS = 1e-4 if USE_AMP and (FLOAT_DTYPE == torch.float16) else 1e-8

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
# 再開モードの場合も, このエポック数の分だけ追加学習される（N_EPOCHSは最終エポック番号ではない）
N_EPOCHS = 10

# 学習時のバッチサイズ
BATCH_SIZE = 100

# データセットの存在するフォルダ・ファイル名
DATA_DIR = './Datasets/'
TRAIN_INPUT_IMAGES_FILE  = 'defacto_train_input_images.pt'
TRAIN_TARGET_IMAGES_FILE = 'defacto_train_target_images.pt'
VALID_INPUT_IMAGES_FILE  = 'defacto_valid_input_images.pt'
VALID_TARGET_IMAGES_FILE = 'defacto_valid_target_images.pt'
TEST_INPUT_IMAGES_FILE   = 'defacto_test_input_images.pt'
TEST_TARGET_IMAGES_FILE  = 'defacto_test_target_images.pt'

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # 入力画像のチャンネル数（カラー画像なら3，グレースケール画像なら1．なお，正解のマスク画像のチャンネル数は常に1）

# 学習結果の保存先フォルダ
MODEL_DIR = './defacto_models/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'forgery_detector_model.pth')

# 中断／再開の際に用いる一時ファイル
CHECKPOINT_EPOCH = os.path.join('./temp/', 'checkpoint_epoch.pkl')
CHECKPOINT_MODEL = os.path.join('./temp/', 'checkpoint_model.pth')
CHECKPOINT_OPT = os.path.join('./temp/', 'checkpoint_opt.pth')

##### ニューラルネットワークモデルの定義

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 畳込み，バッチ正規化，ReLUをセットで行うクラス
class myConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(myConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


# 逆畳込み，バッチ正規化，ReLUをセットで行うクラス
class myConvTranspose2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(myConvTranspose2d, self).__init__()
        self.conv = nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


# defacto の画像中から改ざん領域を推定するニューラルネットワーク
class ForgeryDetector(nn.Module):

    # C: 入力画像のチャンネル数（1または3と仮定）
    # H: 入力画像の縦幅（8の倍数と仮定）
    # W: 入力画像の横幅（8の倍数と仮定）
    def __init__(self, C, H, W):
        super(ForgeryDetector, self).__init__()

        # 畳込み層1
        # カーネルサイズ3，ストライド幅1，パディング1の設定なので，これを通しても特徴マップの縦幅・横幅は変化しない
        self.conv1 = myConv2d(in_channels=C, out_channels=16, kernel_size=3, stride=1, padding=1)

        # 畳込み層2～4
        # カーネルサイズ4，ストライド幅2，パディング1の設定なので，これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 1/2 になる
        # 3つ適用することになるので，最終的には都合 1/8 になる -> ゆえに，入力画像の縦幅と横幅を各々8の倍数と仮定している
        self.conv2 = myConv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv3 = myConv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.conv4 = myConv2d(in_channels=64, out_channels=64, kernel_size=4, stride=2, padding=1)

        # 逆畳込み層5～7
        # カーネルサイズ4，ストライド幅2，パディング1の設定なので，これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 2 倍になる
        # 3つ適用することになるので，最終的には元の大きさに戻る
        self.deconv5 = myConvTranspose2d(in_channels=64, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.deconv6 = myConvTranspose2d(in_channels=128, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.deconv7 = myConvTranspose2d(in_channels=64, out_channels=16, kernel_size=4, stride=2, padding=1)

        # 畳込み層8
        # カーネルサイズ3，ストライド幅1，パディング1の設定なので，これを通しても特徴マップの縦幅・横幅は変化しない
        self.conv8 = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        h1 = self.conv1(x)
        h2 = self.conv2(h1)
        h3 = self.conv3(h2)
        h4 = self.conv4(h3)
        h = self.deconv5(h4)
        h = torch.cat([h, h3], dim=1) # U-net 型の skip connection
        h = self.deconv6(h)
        h = torch.cat([h, h2], dim=1) # U-net 型の skip connection
        h = self.deconv7(h)
        h = torch.cat([h, h1], dim=1) # U-net 型の skip connection
        y = torch.sigmoid(self.conv8(h))
        return y

##### 訓練データセットの読み込み

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from mylib.data_io import TensorDataset


# 再開モードの場合は，前回使用したデータセットをロードして使用する
if RESTART_MODE:

    # テンソルファイルを読み込み，前回使用したデータセットを用意
    train_dataset = TensorDataset(filenames=[
        os.path.join('./temp/', TRAIN_INPUT_IMAGES_FILE),
        os.path.join('./temp/', TRAIN_TARGET_IMAGES_FILE)
    ])
    valid_dataset = TensorDataset(filenames=[
        os.path.join('./temp/', VALID_INPUT_IMAGES_FILE),
        os.path.join('./temp/', VALID_TARGET_IMAGES_FILE)
    ])
    train_size = len(train_dataset)
    valid_size = len(valid_dataset)

# そうでない場合は，データセットを読み込む
else:

    # テンソルファイルを読み込み, 訓練データセットを用意
    dataset = TensorDataset(filenames=[
        os.path.join(DATA_DIR, TRAIN_INPUT_IMAGES_FILE),
        os.path.join(DATA_DIR, TRAIN_TARGET_IMAGES_FILE)
    ])

    # 訓練データセットを分割し，一方を検証用に回す
    dataset_size = len(dataset)
    valid_size = int(0.01 * dataset_size) # 全体の 1% を検証用に
    train_size = dataset_size - valid_size # 残りの 99% を学習用に
    train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

    # データセット情報をファイルに保存
    torch.save(torch.cat([torch.unsqueeze(train_dataset[i][0], dim=0) for i in range(len(train_dataset))], dim=0), os.path.join('./temp/', TRAIN_INPUT_IMAGES_FILE))
    torch.save(torch.cat([torch.unsqueeze(train_dataset[i][1], dim=0) for i in range(len(train_dataset))], dim=0), os.path.join('./temp/', TRAIN_TARGET_IMAGES_FILE))
    torch.save(torch.cat([torch.unsqueeze(valid_dataset[i][0], dim=0) for i in range(len(valid_dataset))], dim=0), os.path.join('./temp/', VALID_INPUT_IMAGES_FILE))
    torch.save(torch.cat([torch.unsqueeze(valid_dataset[i][1], dim=0) for i in range(len(valid_dataset))], dim=0), os.path.join('./temp/', VALID_TARGET_IMAGES_FILE))


# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 性能評価のための関数の定義

In [5]:
import torch


# 二枚の白黒画像の一致度を調べる
def region_consistency_metric(estimated, gt, threshold=0.2):

    def ratio(a, b):
        indices = b.nonzero()
        if len(indices) == 0:
            return 0
        else:
            return float(torch.mean(a[indices] / b[indices]))

    one = torch.ones(gt.size()).to(gt.device)
    zero = torch.zeros(gt.size()).to(gt.device)
    gt = torch.where(gt > threshold, one, zero)
    estimated = torch.where(estimated > threshold, one, zero)
    intersection = estimated * gt
    union = estimated + gt
    union = torch.where(union > 1, one, union)
    E = torch.sum(estimated, dim=(1, 2, 3))
    G = torch.sum(gt, dim=(1, 2, 3))
    I = torch.sum(intersection, dim=(1, 2, 3))
    U = torch.sum(union, dim=(1, 2, 3))
    recall = ratio(I, G)
    precision = ratio(I, E)
    iou = ratio(I, U)

    return recall, precision, iou

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images
from mylib.utility import save_checkpoint, load_checkpoint


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# ニューラルネットワークの作成
model = ForgeryDetector(C=C, H=H, W=W).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters(), eps=ADAM_EPS)

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
if RESTART_MODE:
    INIT_EPOCH, LAST_EPOCH, model, optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, N_EPOCHS, model, optimizer)
    print('')

# 損失関数
loss_func = nn.BCELoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss', 'recall', 'precision', 'IoU'], init_epoch=INIT_EPOCH)

# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader):
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE) # 入力画像
        Y = Y.to(DEVICE) # 正解のマスク画像
        with torch.amp.autocast_mode.autocast(enabled=USE_AMP, device_type='cuda', dtype=FLOAT_DTYPE):
            Y_pred = model(X) # 入力画像 X をニューラルネットワークに入力し，改ざん領域の推測値 Y_pred を得る
        loss = loss_func(Y_pred.to(torch.float32), Y) # 損失関数の現在値を計算
        with torch.amp.autocast_mode.autocast(enabled=USE_AMP, device_type='cuda', dtype=FLOAT_DTYPE):
            LOSS_SCALER.scale(loss).backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
            LOSS_SCALER.step(optimizer)
            LOSS_SCALER.update() # 勾配に沿ってパラメータの値を更新
            sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    sum_recall = 0
    sum_precision = 0
    sum_IoU = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE) # 入力画像
            Y = Y.to(DEVICE) # 正解のマスク画像
            Y_pred = model(X)
            loss = loss_func(Y_pred, Y)
            recall, precision, IoU = region_consistency_metric(Y_pred, Y) # 評価指標の値を計算
            sum_recall += recall * len(X)
            sum_precision += precision * len(X)
            sum_IoU += IoU * len(X)
            sum_loss += float(loss) * len(X)
    avg_recall = sum_recall / valid_size
    avg_precision = sum_precision / valid_size
    avg_IoU = sum_IoU / valid_size
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    loss_viz.add_value('recall', avg_recall) # 検証用データに対する評価指標の値を記録
    loss_viz.add_value('precision', avg_precision) # 同上
    loss_viz.add_value('IoU', avg_IoU) # 同上
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('recall = {0:.6f}'.format(avg_recall))
    print('precision = {0:.6f}'.format(avg_precision))
    print('IoU = {0:.6f}'.format(avg_IoU))
    print('')

    # 学習経過の表示
    if epoch == 0:
        show_images(Y.to('cpu').detach(), num=8, num_per_row=8, title='ground truth', save_fig=False, save_dir=MODEL_DIR)
    show_images(Y_pred.to('cpu').detach(), num=8, num_per_row=8, title='epoch {0}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR)

    # 現在の学習状態を一時ファイルに保存
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, epoch+1, model, optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 必要なら，現在の学習経過情報を自分のgoogle driveに退避
- Google Colab以外の環境で使用することは想定していません．

In [ ]:
import os
from google.colab import drive


# 退避先フォルダの名称
# 自分のgoogle driveにおけるルートフォルダの直下にこの名前のフォルダが作成される
# 既に存在する場合は一度削除した上で再作成される（削除したフォルダはgoogle driveのゴミ箱に移動するようです．必要に応じて完全に削除して下さい）
DST_DIR_NAME = 'AI_advanced_temp'

# tempフォルダをまるごとgoogle driveに退避
MOUNT_POINT = '/content/drive'
DST_DIR_PATH = os.path.join(MOUNT_POINT, 'MyDrive', DST_DIR_NAME)
if not os.path.isdir(MOUNT_POINT):
    drive.mount(MOUNT_POINT)
if os.path.exists(DST_DIR_PATH):
    !rm -r $DST_DIR_PATH
!cp -r temp $DST_DIR_PATH

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = ForgeryDetector(C=C, H=H, W=W)
model.load_state_dict(torch.load(MODEL_FILE, weights_only=True))

##### テストデータセットの読み込み

In [7]:
from torch.utils.data import DataLoader
from mylib.data_io import TensorDataset


# テンソルファイルを読み込み, 訓練データセットを用意
if RESTART_MODE:
    test_dataset = TensorDataset(filenames=[
        os.path.join('./temp/', TEST_INPUT_IMAGES_FILE),
        os.path.join('./temp/', TEST_TARGET_IMAGES_FILE)
    ])
else:
    test_dataset = TensorDataset(filenames=[
        os.path.join(DATA_DIR, TEST_INPUT_IMAGES_FILE),
        os.path.join(DATA_DIR, TEST_TARGET_IMAGES_FILE)
    ])
test_size = len(test_dataset)

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### テスト処理

In [ ]:
import torch
from mylib.data_io import show_images


model = model.to(DEVICE)
model.eval()

# テストデータセットで精度を評価
sum_recall = 0
sum_precision = 0
sum_IoU = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE) # 入力画像
        Y = Y.to(DEVICE) # 正解のマスク画像
        with torch.amp.autocast_mode.autocast(enabled=USE_AMP, device_type='cuda', dtype=FLOAT_DTYPE):
            Y_pred = model(X)
        recall, precision, IoU = region_consistency_metric(Y_pred.to(torch.float32), Y)
        sum_recall += recall * len(X)
        sum_precision += precision * len(X)
        sum_IoU += IoU * len(X)
avg_recall = sum_recall / test_size
avg_precision = sum_precision / test_size
avg_IoU = sum_IoU / test_size
print('recall = {0:.6f}'.format(avg_recall))
print('precision = {0:.6f}'.format(avg_precision))
print('IoU = {0:.6f}'.format(avg_IoU))
print('')

# 推定結果の例を表示
show_images(Y.to('cpu').detach(), num=32, num_per_row=8, title='ground truth', save_fig=False, save_dir=MODEL_DIR)
show_images(Y_pred.to('cpu').detach(), num=32, num_per_row=8, title='estimated', save_fig=False, save_dir=MODEL_DIR)